# Model that has a prompt, takes in a situation and a questions and returns relevant articles to answer the question

## Input Situation & Question

In [135]:
situation = "Legal Situation: Hr. X war seit sechs Jahren aufgrund eines mündlich geschlossenen Arbeitsvertrages bei der Firma Y AG tätig. \
Vor einigen Monaten erhielt er eine neue Chefin, mit der er sich offenbar nicht verstand. Angesichts der zunehmenden Verschlechterung \
seines Verhältnisses zur Vorgesetzten sah die Y AG keinen anderen Ausweg mehr, als sich von X zu trennen. Am Dienstag, 24. Februar 2015, \
sandte sie X mit eingeschriebener Post ein Kündigungsschreiben. Da X beim Zustellversuch vom Mittwoch, 25. Februar, nicht zu Hause war, \
wurde eine Abholungseinladung in seinen Briefkasten gelegt. In der Folge holte X die Sendung bei der Post ab, dies allerdings erst am \
nächsten Montag, dem 2. März. Im Kündigungsschreiben hielt die Y AG fest, dass sie das Arbeitsverhältnis auf den nächstmöglichen Kündigungstermin \
auflösen wollte."

question = "Fragen: Analysiere und kläre, wann genau das Arbeitsverhältnis von X mit der Y AG endet. Es stellen sich die folgenden Fragen: \
A) Welche Kündigungsfrist gilt? \
B) In welchem Zeitpunkt wurde die Kündigung wirksam? \
C) Wann endet folglich das Arbeitsverhältnis?"

prompt = "You are a smart and helpful assistant to a lawyer in Switzerland. You have a client for whom you need to create an analysis of the legal situation.\
You will always answer in German. And you will always look up the paragraphs for correct recitation.\
You will perform your task as follows: \
1. you pick the first question of the user and you will determine, which law books, articles and paragraphs are relevant. \
2. you will retrieve the relevant paragraphs from the law books. \
3. you will review all retrieved paragraphs and you will determine which of them are relevant. you will keep only the relevant ones. \
4. you will return the relevant paragraphs to the user in the defined format below: \
question | paragraph number | paragraph citation"


## Let's start with creating the functions

In [2]:
import os
import logging
import openai
import requests
from pydantic import BaseModel, Field
from typing import Optional

import datetime
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

#debug
%pdb off

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.agents import tool

lm_name =  "gpt-4-1106-preview" #"gpt-3.5-turbo-0613" #"gpt-4-0314"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "lawassist"

Automatic pdb calling has been turned OFF


In [3]:
# Define the input schema
class ParagraphFinder(BaseModel):
    articleNum: int = Field(..., description="The article number to search for. Must be specified.")
    articleNumMinor: Optional[str] = Field(None, description="The minor version of the article to search for. E.g. in Art. 123b it would be 'b'")
    paragraphNum: Optional[int] = Field(None, description="The paragraph to search for. Provide only the number of the Paragraph, without any letters like 'bis' or 'ter'. If not specified, all paragraphs of the article are returned as a list.")
    lawbook: str = Field(..., description="The lawbook to search for. Currently supported is OR and ZGB. This parameter must be supplied.")
 

@tool(args_schema=ParagraphFinder)
def get_paragraphs(lawbook: str, articleNum: int, articleNumMinor: str = None, paragraphNum: int = None) -> dict:
    """Fetch one or more paragraphs for selected lawbook, article number and paragraph. if an optional parameter is not supplied, all paragraphs of the article are returned. The articleNum and the paragraphNum must be a integer. E.g. Art. 335c OR will be called as lawbook="OR" articleNum=335 articleNumMinor="c" paragraphNum=None."""
    
    BASE_URL = "http://127.0.0.1:8000/lawbook/getParagraphs"
    
    # Parameters for the request
    params = {
        'bookName': lawbook,
        'articleNum': articleNum,
        'articleNumMinor': articleNumMinor,
        'paragraphNum' : paragraphNum,
    }

    # Make the request
    print(f"Making request to {BASE_URL} with params: {params}")
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    return results
    

In [40]:
tools = [get_paragraphs,]

In [89]:
functions = [
    format_tool_to_openai_function(f) for f in [
        get_paragraphs
    ]
]
model = ChatOpenAI(temperature=0, model=lm_name, response_format={ "type": "json_object" }).bind(functions=functions)

WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.


In [122]:
print(functions)

[{'name': 'get_paragraphs', 'description': 'get_paragraphs(lawbook: str, articleNum: int, articleNumMinor: str = None, paragraphNum: int = None) -> dict - Fetch all paragraph for selected lawbook, article number and paragraph. if an optional parameter is not supplied, all paragraphs of the article are returned. The articleNum and the paragraphNum must be a integer. E.g. Art. 335c OR will be called as lawbook="OR" articleNum=335 articleNumMinor="c" paragraphNum=None.', 'parameters': {'title': 'ParagraphFinder', 'type': 'object', 'properties': {'articleNum': {'title': 'Articlenum', 'description': 'The article number to search for. Must be specified.', 'type': 'integer'}, 'articleNumMinor': {'title': 'Articlenumminor', 'description': "The minor version of the article to search for. E.g. in Art. 123b it would be 'b'", 'type': 'string'}, 'paragraphNum': {'title': 'Paragraphnum', 'description': "The paragraph to search for. Provide only the number of the Paragraph, without any letters like '

In [118]:
# define the prompt
from langchain.prompts import MessagesPlaceholder
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


In [85]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "get_paragraphs": get_paragraphs, 
        }
        return tools[result.tool].run(result.tool_input)

In [64]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [125]:
result = chain.invoke({"input": situation + question, 
                      "agent_scratchpad": []})
print("Result: ", result)

Result:  tool='get_paragraphs' tool_input={'lawbook': 'OR', 'articleNum': 335, 'articleNumMinor': 'c'} log="\nInvoking: `get_paragraphs` with `{'lawbook': 'OR', 'articleNum': 335, 'articleNumMinor': 'c'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_paragraphs', 'arguments': '{"lawbook":"OR","articleNum":335,"articleNumMinor":"c"}'}})]


### Now let's parse out the results and give it back

In [103]:
from langchain.output_parsers import StructuredOutputParser
import pandas as df


def parse_law_data(json_data)->list:
    parsed_data = []
    for entry in json_data:
        row = {
            'Lawbook': entry['bookName'],
            'Article Number': entry['articleNum'],
            'Article Number Minor': entry.get('articleNumMinor', ''),
            'Paragraph Number': entry.get('paragraphNum', ''),
            'Paragraph Text': entry['paragraphText']
        }
        parsed_data.append(row)
    return parsed_data


table_data = parse_law_data(result)
data = df.DataFrame(table_data)
#print all data in a table 'data' in a pretty format
data


TypeError: tuple indices must be integers or slices, not str

### With Conversational Memory

In [104]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [105]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [68]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.runnable import RunnablePassthrough

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [69]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "get_paragraphs": get_paragraphs 
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [136]:
result2 = run_agent(situation + question)

Making request to http://127.0.0.1:8000/lawbook/getParagraphs with params: {'bookName': 'OR', 'articleNum': 335, 'articleNumMinor': 'c', 'paragraphNum': None}
Making request to http://127.0.0.1:8000/lawbook/getParagraphs with params: {'bookName': 'OR', 'articleNum': 335, 'articleNumMinor': None, 'paragraphNum': None}
Making request to http://127.0.0.1:8000/lawbook/getParagraphs with params: {'bookName': 'OR', 'articleNum': 336, 'articleNumMinor': None, 'paragraphNum': None}
Making request to http://127.0.0.1:8000/lawbook/getParagraphs with params: {'bookName': 'OR', 'articleNum': 337, 'articleNumMinor': None, 'paragraphNum': None}


In [137]:
result2.return_values['output']

'```json\n{\n  "Reference to relevant question": 1,\n  "Relevant law": [\n    {\n      "Law citation": "OR Art. 335 Abs. 1"\n    },\n    {\n      "Law citation": "OR Art. 335c Abs. 1-3"\n    },\n    {\n      "Law citation": "OR Art. 336 Abs. 1-3"\n    },\n    {\n      "Law citation": "OR Art. 336c Abs. 1-3"\n    },\n    {\n      "Law citation": "OR Art. 337 Abs. 1-3"\n    }\n  ]\n}\n```'

In [138]:
import json
import pandas as pd

# Assuming this is the string extracted from the result2.return_values['output']
output_string = result2.return_values['output']

# Strip the Markdown code block syntax (triple backticks and 'json' keyword)
json_string = output_string.strip('`').strip('json\n')

# Parse the JSON string into a Python dictionary
parsed_json = json.loads(json_string)

# Create a pandas DataFrame from the entire JSON object, not just the 'Relevant law' part
df = pd.DataFrame(parsed_json)

# Display the DataFrame including all elements
print(df)


   Reference to relevant question                               Relevant law
0                               1     {'Law citation': 'OR Art. 335 Abs. 1'}
1                               1  {'Law citation': 'OR Art. 335c Abs. 1-3'}
2                               1   {'Law citation': 'OR Art. 336 Abs. 1-3'}
3                               1  {'Law citation': 'OR Art. 336c Abs. 1-3'}
4                               1   {'Law citation': 'OR Art. 337 Abs. 1-3'}


### Create a chatbot

In [77]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param


class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [78]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

NameError: name 'AgentExecutor' is not defined